# **# Big Data Platforms Final Project: Small Files Problem in MapReduce**


**Authors:** 
 
Meital Avitan, 204945323 

Oren Avidan, 304823735

<br>

# **# Lithops & IBM Cloud Setup**

In this section, we set up Lithops & IBM cloud for the demonstration.



In [1]:
# installing lithops
! pip install lithops

     |████████████████████████████████| 308 kB 5.7 MB/s 
     |████████████████████████████████| 146 kB 48.8 MB/s 
     |████████████████████████████████| 173 kB 41.5 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 210 kB 24.9 MB/s 
     |████████████████████████████████| 1.5 MB 38.4 MB/s 
     |████████████████████████████████| 154 kB 36.0 MB/s 
     |████████████████████████████████| 225 kB 40.1 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 913 kB 11.1 MB/s 
     |████████████████████████████████| 135 kB 44.5 MB/s 
     |████████████████████████████████| 63 kB 201 kB/s 
INFO: pip is looking at multiple versions of ibm-cos-sdk to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 52 kB 551 kB/s 
     |████████████████████████████████| 888 kB 40.4 MB/s 
     |████████████████████████████████| 133 kB 40

In [2]:
# testing lithops
! lithops test

2022-02-15 15:52:03,923 [INFO] lithops.config -- Lithops v2.5.8
2022-02-15 15:52:03,925 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-02-15 15:52:03,925 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-02-15 15:52:03,925 [INFO] lithops.invokers -- ExecutorID 45740a-0 | JobID A000 - Selected Runtime: python3 
2022-02-15 15:52:03,926 [INFO] lithops.invokers -- Runtime python3 is not yet installed
2022-02-15 15:52:03,926 [INFO] lithops.localhost.localhost -- Extracting preinstalled Python modules from python3
2022-02-15 15:52:04,573 [INFO] lithops.invokers -- ExecutorID 45740a-0 | JobID A000 - Starting function invocation: hello() - Total: 1 activations
2022-02-15 15:52:04,574 [INFO] lithops.invokers -- ExecutorID 45740a-0 | JobID A000 - View execution logs at /tmp/lithops/logs/45740a-0-A000.log
2022-02-15 15:52:04,580 [INFO] lithops.wait -- ExecutorID 45740a-0 - Getting results from functions

100% 1/1 [00:02<00:00

In [3]:
# importing relevant packages

# importing lithops
import lithops
from lithops import FunctionExecutor
from lithops import Storage

# general
import os
import io
import time
import random
import warnings
import threading
import sqlite3
import shutil

# ml
import numpy as np
import scipy as sp
import pandas as pd

# visual
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display

# importing time measuring tools
from timeit import timeit
import time

In [4]:
# configuring connection to IBM cloud

endpoint = 'https://eu-gb.functions.cloud.ibm.com'
namespace = 'oren.avidan@post.idc.ac.il_dev'
api_key = '2f19a215-3fc6-492a-8329-8e51ea720d6c:n49jeGesU220XDcxk33JK65FlerI4DQ2N9IvVVjrnBX1zhojoWtWbT4LIPFOOh31'
storage_bucket = 'big-data-project-oren-meital'
region = 'eu-de'
access_key = "b8a563ccf8644ed6ad152140d2af83f1" 
secret_key = "264718d46208e4dd7a8c4b9e8720a3dcc182aa2e2c9b5c56"

In [5]:
# configuring lithops against IBM cloud

config = {'lithops': {'storage': 'ibm_cos'},

          'ibm_cf':  {'endpoint': endpoint,
                      'namespace': namespace,
                      'api_key': api_key},

          'ibm_cos': {'storage_bucket': storage_bucket,
                      'region': region,
                      'access_key' : access_key ,
                      'secret_key' : secret_key }}


In [6]:
# testing remote connection to IBM cloud (remote test) using lithops

def hello_world(name):
    return 'Hello {}!'.format(name)

if __name__ == '__main__':
    fexec = lithops.FunctionExecutor(config=config)
    fexec.call_async(hello_world, 'World')
    print(fexec.get_result())

2022-02-15 15:52:18,046 [INFO] lithops.config -- Lithops v2.5.8
2022-02-15 15:52:18,164 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS client created - Region: eu-de
2022-02-15 15:52:18,166 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: eu-gb - Namespace: oren.avidan@post.idc.ac.il_dev
2022-02-15 15:52:18,167 [INFO] lithops.invokers -- ExecutorID 0b462f-0 | JobID A000 - Selected Runtime: lithopscloud/ibmcf-python-v37 - 256MB
2022-02-15 15:52:19,145 [INFO] lithops.invokers -- ExecutorID 0b462f-0 | JobID A000 - Starting function invocation: hello_world() - Total: 1 activations
2022-02-15 15:52:19,148 [INFO] lithops.invokers -- ExecutorID 0b462f-0 | JobID A000 - View execution logs at /tmp/lithops/logs/0b462f-0-A000.log
2022-02-15 15:52:19,152 [INFO] lithops.wait -- ExecutorID 0b462f-0 - Getting results from functions


    0%|          | 0/1  

2022-02-15 15:52:22,368 [INFO] lithops.executors -- ExecutorID 0b462f-0 - Cleaning temporary data



Hello World!


In [7]:
# setting storage configuration for accessing files on cloud
storage = Storage(config=config)


2022-02-15 15:52:22,460 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS client created - Region: eu-de


In [8]:
# create and connect to a new sqlite database
conn = sqlite3.connect('my_db') 
c = conn.cursor()

# create temp results table
c.execute('''
          CREATE TABLE IF NOT EXISTS temp_results (
          key TEXT, 
          value TEXT
          );'''
         )

In [9]:
# defining aggregator switch which is True when there are small files
aggregator_switch = False

# **#  Aggregator & MapReduce**
This section includes the main functions of this demonstration.

In [10]:
# Define the minimum file size cut-off, by which we decide a file is "small".
# Ideally, one would choose at least 64MB to simulate the size of a HDFS DataNode.
# Here, for our mini-simulation, we chose a smaller cut-off.

MIN_SIZE = 3.69e3

In [11]:
def aggregator():
    """
    Function that gets data from the cloud, iterates over each file, and 
    aggregates any small files into large ones, according to the globally 
    defined variable MIN_SIZE.

    :return: a list of the aggregated file names
    :rtype: list(basestring)
    """

    # initialize a new file for aggregated small files
    with open('tmp.csv', 'w'):
        pass

    # creating a df that will hold the concatenated file, and then will be saved into csv
    agg_df = pd.DataFrame(columns = ['firstname', 'secondname', 'city', 'originalfile'])
    objects = storage.list_objects(bucket=storage_bucket, prefix='myCSV')
    i = 0
    aggregated_files= []

    for obj in objects:
        key = obj['Key']
        size = obj['Size']

        # if the size of the checked file is smaller than the defined size (64mb) we will add it to the aggregated file
        # other wise it will be left to be handled by itself
        if size < MIN_SIZE:
                        
            # get the data from the cloud in a df format
            data = storage.get_object(bucket=storage_bucket, key=key, stream=False).decode("utf-8")
            data = data.replace('\r', '')
            df = pd.read_csv(io.StringIO(data), lineterminator='\n', header=0)  
            
            # adding another column to the df with the name of the original file to keep it
            df['originalfile'] = key 
            # to change back if we want to keep original files' names
                        
            # add the df to the agg df 
            agg_df = agg_df.append(df, ignore_index=True)
            
            # check the tmp file size
            curr_size = os.path.getsize('tmp.csv')
            
            # save the df as a tmp.csv
            csv = agg_df.to_csv('tmp.csv', mode='w', index=False)
            
            # if the tmp file is still small enough, then we want to keep adding content to the current aggregated file
            if curr_size < MIN_SIZE:
                # instead of appending to tmp, save agg_df as tmp
                csv = agg_df.to_csv('tmp.csv', mode='w', index=False)

            # otherwise, save the current aggregated file and make a new empty tmp
            else:
                aggregated = pd.read_csv('tmp.csv')
                aggregated.to_csv(f'aggregated-{i}.csv', mode='w', index=False)
                aggregated_files.append(f'aggregated-{i}.csv')
                i += 1
                with open('tmp.csv', 'w'):
                    pass
                
                # reseting aggregated 
                agg_df = pd.DataFrame(columns = ['firstname', 'secondname', 'city', 'originalfile'])
                csv = agg_df.to_csv('tmp.csv', mode='w', index=False)

    aggregated_files.append(f'aggregated-{i}.csv')
    aggregated.to_csv(f'aggregated-{i}.csv', mode='w', index=False)

    return aggregated_files

In [12]:
agg_files = aggregator()
print(f'Created aggregated files: {agg_files}')

Created aggregated files: ['aggregated-0.csv', 'aggregated-1.csv']


In [13]:
def inverted_map(document_name):
    """
    Receives a name of a CSV file (firstname, secondname, city) and returns a list that contains entries of the form (key_value, document_name).
    Recognizes special aggregated files and handles them accordingly.
    Returns a list of the form [(key, doc name),(key, doc name),...]

    :param document_name: the name of the csv file to process
    :type document_name: basestring
    :return: the inverted list of data
    :rtype: list of tuples of strings
    """
    if aggregator_switch == True:
        data = pd.read_csv(document_name, index_col=False)
        res = []

        agg_firstname = pd.DataFrame(columns = ['key', 'document_name'])
        agg_secondname = pd.DataFrame(columns = ['key', 'document_name'])
        agg_city = pd.DataFrame(columns = ['key', 'document_name'])

        agg_firstname['key']= 'firstname_' + data['firstname']
        agg_firstname['document_name']= data['originalfile']

        agg_secondname['key']= 'secondname_' + data['secondname']
        agg_secondname['document_name']= data['originalfile']

        agg_city['key']= 'city_' + data['city']
        agg_city['document_name']= data['originalfile']
        agg_converted = pd.concat([agg_firstname, agg_secondname, agg_city], ignore_index=True, axis=0)
        records = agg_converted.values.tolist()
        res = list(records)        

    else:
        data = pd.read_csv(document_name, index_col=False) 
        res=[]
        for col, items in data.iteritems():
            for item in items:
                res.append((col + '_' + item, document_name))

    return res

In [14]:
def inverted_reduce(data):
    """
    Receives a list of the form ['value', 'documents...'] where documents contains duplicate csv names.
    Returns a list of the form ['value', 'documents...'] with value and documents where documents contains no duplicates

    :param data: the values and corresponding documents to process
    :type data: list of tuples, first item being the value, second being the CSVs
    :return: list of the form ['value', 'documents...'] with value and documents where documents contains no duplicates
    :rtype: list of strings
    """
    res = []
    key, docs = data
    unique_documents = ', '.join(list(set(docs.split(','))))
    res.append([key, unique_documents])
    return res

In [15]:
class MapReduceServerlessEngine():
    """
    Class that executes MapReduce based on lithops for map and reduce functions.
    """
    def __init__(self):
        pass

    def execute (self, iterdata, map_function, reduce_function):
        # map stage
        print ('Start of mapping stage')
        fexec = lithops.FunctionExecutor()
        fexec.map(map_function = map_function, map_iterdata = iterdata)
        mapped_data = fexec.get_result()
        
        # storing the results in the same SQLite as we used in MapReduceEngine
        for seq in mapped_data:
            data = pd.DataFrame(seq, columns =['key', 'value'])
            data.to_sql('temp_results', conn, if_exists='append', index=False)

        table_before_shuffling_sorting = c.execute('''
                      SELECT *
                      FROM temp_results;
                      '''
                    ).fetchall()

        sorted_shuffled_data = c.execute('''
                      SELECT key,
                      GROUP_CONCAT(value) AS valueconcat
                      FROM temp_results
                      GROUP BY key 
                      ORDER BY key asc;
                      '''
                    ).fetchall()
        

        # converting from list of tuples to list of lists 
        #(due to a limitation of the map func of lithops)
        sorted_shuffled_data = [list(seq) for seq in sorted_shuffled_data]
        
        # reduce stage
        print ('Start of reducing stage')
        fexec2 = lithops.FunctionExecutor()
        fexec2.map(map_function = reduce_function , map_iterdata = sorted_shuffled_data)
        reduced_data = fexec2.get_result()
        reduced_data = np.squeeze(reduced_data)

        # Return results of reduce tasks to the user
        return reduced_data

In [16]:
def get_data_from_cloud (storage_bucket):
    """
    :param storage_bucket: the cloud storage bucket
    """
    # getting data from cloud and creating an iterable list that includes the 
    # names of the files we want to iter on later
    iterdata = storage.list_keys(bucket = storage_bucket, prefix = 'myCSV')

    for file_name in iterdata :
        # get csv file from cloud
        if __name__ == "__main__":
            data = storage.get_object(bucket = storage_bucket, key = file_name, stream=False).decode("utf-8")
            data = data.replace('\r', '')
        # converting data to df
        df = pd.read_csv(io.StringIO(data), lineterminator='\n', header=0)
        # saving pd to csv on local computer for further work
        df.to_csv(file_name, index=False)

    return iterdata

# **# Demonstration**

Here, we demonstrate how using the aggregator on the small files before doing MapReduce shortens the execution time.

In [17]:
# getting a list of files
input_data = get_data_from_cloud(storage_bucket)

Eexecuting MapReduce without small file aggregation

In [18]:
# NO AGGREGATION

mapreduce = MapReduceServerlessEngine()

# executing
start_time=time.perf_counter()
results = mapreduce.execute(input_data, inverted_map, inverted_reduce)
end_time=time.perf_counter()

time_without_aggregation = end_time - start_time

2022-02-15 15:52:29,591 [INFO] lithops.config -- Lithops v2.5.8
2022-02-15 15:52:29,598 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-02-15 15:52:29,615 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-02-15 15:52:29,618 [INFO] lithops.invokers -- ExecutorID 0b462f-1 | JobID M000 - Selected Runtime: python3 
2022-02-15 15:52:29,647 [INFO] lithops.invokers -- ExecutorID 0b462f-1 | JobID M000 - Starting function invocation: inverted_map() - Total: 20 activations


Start of mapping stage


2022-02-15 15:52:29,803 [INFO] lithops.invokers -- ExecutorID 0b462f-1 | JobID M000 - View execution logs at /tmp/lithops/logs/0b462f-1-M000.log
2022-02-15 15:52:29,832 [INFO] lithops.wait -- ExecutorID 0b462f-1 - Getting results from functions


    0%|          | 0/20  

2022-02-15 15:52:38,220 [INFO] lithops.executors -- ExecutorID 0b462f-1 - Cleaning temporary data


2022-02-15 15:52:38,750 [INFO] lithops.config -- Lithops v2.5.8
2022-02-15 15:52:38,752 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-02-15 15:52:38,754 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-02-15 15:52:38,757 [INFO] lithops.invokers -- ExecutorID 0b462f-2 | JobID M000 - Selected Runtime: python3 
2022-02-15 15:52:38,767 [INFO] lithops.invokers -- ExecutorID 0b462f-2 | JobID M000 - Starting function invocation: inverted_reduce() - Total: 26 activations
2022-02-15 15:52:38,833 [INFO] lithops.invokers -- ExecutorID 0b462f-2 | JobID M000 - View execution logs at /tmp/lithops/logs/0b462f-2-M000.log
2022-02-15 15:52:38,843 [INFO] lithops.wait -- ExecutorID 0b462f-2 - Getting results from functions


Start of reducing stage



    0%|          | 0/26  

2022-02-15 15:52:40,966 [INFO] lithops.executors -- ExecutorID 0b462f-2 - Cleaning temporary data


In [19]:
print('Total time was', time_without_aggregation, 'seconds')

Total time was 11.386754593999996 seconds


Executing MapReduce with small file aggregation

In [20]:
# WITH AGGREGATION

mapreduce = MapReduceServerlessEngine()

# aggregate small files to large files
aggregated_file_names = aggregator()
print(f'agg filenames {aggregated_file_names}')

# executing
start_time=time.perf_counter()
results = mapreduce.execute(aggregated_file_names, inverted_map, inverted_reduce)
end_time=time.perf_counter()

2022-02-15 15:52:43,975 [INFO] lithops.config -- Lithops v2.5.8
2022-02-15 15:52:43,980 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-02-15 15:52:43,984 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-02-15 15:52:43,987 [INFO] lithops.invokers -- ExecutorID 0b462f-3 | JobID M000 - Selected Runtime: python3 
2022-02-15 15:52:43,998 [INFO] lithops.invokers -- ExecutorID 0b462f-3 | JobID M000 - Starting function invocation: inverted_map() - Total: 2 activations
2022-02-15 15:52:44,068 [INFO] lithops.invokers -- ExecutorID 0b462f-3 | JobID M000 - View execution logs at /tmp/lithops/logs/0b462f-3-M000.log
2022-02-15 15:52:44,083 [INFO] lithops.wait -- ExecutorID 0b462f-3 - Getting results from functions


agg filenames ['aggregated-0.csv', 'aggregated-1.csv']
Start of mapping stage



    0%|          | 0/2  

2022-02-15 15:52:46,203 [INFO] lithops.executors -- ExecutorID 0b462f-3 - Cleaning temporary data
2022-02-15 15:52:46,254 [INFO] lithops.config -- Lithops v2.5.8
2022-02-15 15:52:46,257 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-02-15 15:52:46,260 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-02-15 15:52:46,264 [INFO] lithops.invokers -- ExecutorID 0b462f-4 | JobID M000 - Selected Runtime: python3 
2022-02-15 15:52:46,277 [INFO] lithops.invokers -- ExecutorID 0b462f-4 | JobID M000 - Starting function invocation: inverted_reduce() - Total: 38 activations
2022-02-15 15:52:46,343 [INFO] lithops.invokers -- ExecutorID 0b462f-4 | JobID M000 - View execution logs at /tmp/lithops/logs/0b462f-4-M000.log
2022-02-15 15:52:46,361 [INFO] lithops.wait -- ExecutorID 0b462f-4 - Getting results from functions



Start of reducing stage



    0%|          | 0/38  

2022-02-15 15:52:48,574 [INFO] lithops.executors -- ExecutorID 0b462f-4 - Cleaning temporary data


In [21]:
time_with_aggregation = end_time - start_time
print('Total time was', time_with_aggregation, 'seconds')

Total time was 4.605028007000016 seconds


In this small demonstration, we saw that running MapReduce with aggregation of the small files is faster than without.